In [1]:
# Importing the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


**Regression**

Выходные данные непрерывны, например, цена дома

[микрорайон, площадь, этаж, и др.] –> нейронная сеть –> [цена дома]

In [2]:
# Creating a Regressor class
class Regressor(torch.nn.Module):
    def __init__(self, inputs):
        super().__init__()        
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(inputs, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1),
            torch.nn.ReLU()
        )

    def forward(self, x):
        return self.encoder(x)

In [3]:
actvation = 'relu' # (rectified linear unit)
loss = 'MSE' # mean square error
optimizer = 'rmsprop' # обратное распространение корня квадратного из среднеквадратичной ошибки

$$
F(x) = max(0, x)
$$

<img src="images/ReLU.png" alt="ReLU" height=50% width=50%>

**Binary classification** (logistic regression)

Выходные данные могут иметь два состояния или класса: да/нет, истина/ложь, 0/1 и так далее.

(Пасажиры Титаника) [Возраст, Пол, Класс, и др.] –> нейронная сеть –> [Выживет/Не выживет]

In [4]:
# Creating a BinaryClassifier class
class BinaryClassifier(torch.nn.Module):
    def __init__(self, inputs):
        super().__init__()        
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(inputs, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        return self.encoder(x)

In [5]:
actvation = 'sigmoid'
loss = 'binary_crossentropy'
optimizer = 'rmsprop'
metrics = 'accuracy'

$$
F(x) = {1 \over 1 + e^{-x}}
$$

<img src="images/Sigmoid.png" alt="Sigmoid" height=50% width=50%>

**Multiclass classification** (multinominal logistic regression)

Допустим, у нас есть набор антропометрических показателей (к примеру, рост, вес и пол), и мы хотим предсказывать, является ли некто **новорожденным, малышом, ребёнком, подростком или взрослым**. Мы хотим, чтобы наша модель классифицировала или предсказывала более чем из одного класса или метки – в данном примере у нас в сумме пять классов возрастных категорий. 

[рост, вес, пол, и др.] –> нейронная сеть –> [подросток]

In [6]:
# Creating a MulticlassClassifier class
class MulticlassClassifier(torch.nn.Module):
    def __init__(self, inputs, output):
        super().__init__()        
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(inputs, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, output),
            torch.nn.Softmax()
        )

    def forward(self, x):
        return self.encoder(x)

In [7]:
actvation = 'softmax'
loss = 'categorical_crossentropy' # в отличии от бинарного классификатора
optimizer = 'adam' # rmsprop + adagrad (адаптивный градиент), а также адаптивная скорость обучения
metrics = 'accuracy'

Softmax применяется в тех случаях, когда необходимо, чтобы сумма элементов была равно 1, а каждый элемент принадлежал интервалу [0; 1]

$$
F(x) = {e^{x} \over \sum_j e^{x_j}}
$$

<img src="images/softmax.png" alt="softmax" height=70% width=70%>

**Multi-label multi-class classification**

На этот раз мы удалим пол как один из признаков и вместо этого сделаем его одной из меток, которую будем предсказывать. 

[рост, вес, и др.] –> нейронная сеть –> [подросток, женского пола]

Число выходных классов в выходном слое равно сумме всех выходных категорий. В данном случае раньше у нас их было пять, а теперь мы добавляем еще два для пола, в общей сложности семь. Мы также хотим трактовать каждый выходной класс как двоичный классификатор, и, стало быть, нам нужен ответ типа«да/нет», поэтому мы меняем активационную функцию на сигмоиду.

In [8]:
# Creating a Multi-label MulticlassClassifier class
class MultilabelMulticlassClassifier_1(torch.nn.Module):
    def __init__(self, inputs, output):
        super().__init__()        
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(inputs, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, output),
            torch.nn.Sigmoid()
        )


    def forward(self, x):
        return self.encoder(x)

In [9]:
actvation = 'sigmoid'
loss = 'binary_crossentropy'
optimizer = 'rmsprop'
metrics = 'accuracy'

**Потенциальная проблема:**
Давайте допустим, что мы выдаем два класса с самыми высокими значениями (от 0 до 1); то есть наиболее уверенные предсказания. Что делать, если в результате нейронная сеть с высокой
уверенностью предсказывает и взрослого, и подростка, а также с меньшей уверенностью мужской пол, и женский пол? 

**Добавим постобработку -**
выбираем наивысшую уверенность из первых пяти выходных классов (возраст) и наивысшую уверенность из последних
двух классов (пол). Другими словами, мы делим семь выходных классов на две соответствующие категории и из каждой категории выбираем выход с наивысшим уровнем уверенности.

```python
mm_classifier = MultilabelMulticlassClassifier_1(inputs, output)
y = mm_classifier(x)
age = max(y[:5])
sex = max(y[-2:])
```

Другой способ решить проблему:

<img src="images/MultilabelMulticlassClassifier.png" alt="MultilabelMulticlassClassifier" height=70% width=70%>

Поскольку каждый выходной слой делает независимые предсказания, мы можем вернуться к их трактовке как мультиклассового классификатора – и, стало быть, мы возвращаемся к использованию 'softmax' в качестве активационной функции, 'categorical_crossentropy' в качестве функции потери и 'adam' в качестве оптимизатора.

In [10]:
# Creating a Multi-label MulticlassClassifier class
class MultilabelMulticlassClassifier_2(torch.nn.Module):
    def __init__(self, inputs, output1, output2):
        super().__init__()        
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(inputs, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 64),
            torch.nn.ReLU(),
        )
        self.output1 = torch.nn.Sequental(
            torch.nn.Linear(64, output1),
            torch.nn.Softmax()
        )
        self.output2 = torch.nn.Sequental(
            torch.nn.Linear(64, output2),
            torch.nn.Softmax()
        )

    def forward(self, x):
        encode = self.encoder(x)
        out1 = self.output1(encode)
        out2 = self.output2(encode)
        return torch.cat((out1, out2), dim=0)

In [11]:
actvation = 'softmax', 'softmax'
loss = 'categorical_crossentropy'
optimizer = 'adam' 
metrics = 'accuracy'